In [1]:
import os
import numpy as np

import scipy.io as scio

You have to donwload the dataset and define the path below.

In [2]:
file_path = '/data/SEED_EEG/Preprocessed_EEG/'

In [3]:
all_file = []
for file in os.listdir(file_path):
    if file[-3:] == 'mat':
        all_file.append(file)
all_file.sort()
all_file

['10_20131130.mat',
 '10_20131204.mat',
 '10_20131211.mat',
 '11_20140618.mat',
 '11_20140625.mat',
 '11_20140630.mat',
 '12_20131127.mat',
 '12_20131201.mat',
 '12_20131207.mat',
 '13_20140527.mat',
 '13_20140603.mat',
 '13_20140610.mat',
 '14_20140601.mat',
 '14_20140615.mat',
 '14_20140627.mat',
 '15_20130709.mat',
 '15_20131016.mat',
 '15_20131105.mat',
 '1_20131027.mat',
 '1_20131030.mat',
 '1_20131107.mat',
 '2_20140404.mat',
 '2_20140413.mat',
 '2_20140419.mat',
 '3_20140603.mat',
 '3_20140611.mat',
 '3_20140629.mat',
 '4_20140621.mat',
 '4_20140702.mat',
 '4_20140705.mat',
 '5_20140411.mat',
 '5_20140418.mat',
 '5_20140506.mat',
 '6_20130712.mat',
 '6_20131016.mat',
 '6_20131113.mat',
 '7_20131027.mat',
 '7_20131030.mat',
 '7_20131106.mat',
 '8_20140511.mat',
 '8_20140514.mat',
 '8_20140521.mat',
 '9_20140620.mat',
 '9_20140627.mat',
 '9_20140704.mat',
 'label.mat']

In [14]:
# 1. Divide the dataset into train/valid/test with ratio of 3:1:1
# train_idx = [2, 4, 5, 7, 8, 9, 11, 13, 15]
# valid_idx = [1, 6, 14]
# test_idx = [3, 10, 12]

train_idx = list(np.random.choice(15, 9, replace=False))
valid_idx = list(np.random.choice(6, 3, replace=False))
train_idx.sort()
valid_idx.sort()

print(train_idx)
print(valid_idx)

[1, 3, 4, 6, 7, 8, 10, 12, 14]
[2, 3, 4]


In [11]:
train_idx = [2, 4, 5, 7, 8, 10, 11, 12, 15]
valid_idx = [1, 3, 14]
test_idx = [6, 9, 13]

In [12]:
train_file = []
valid_file = []
test_file = []

for file in all_file:
    if file == 'label.mat':
        continue
    patient_idx = int(file.split('_')[0])
    if patient_idx in train_idx:
        train_file.append(file)
    elif patient_idx in valid_idx:
        valid_file.append(file)
    else:
        test_file.append(file)

In [6]:
# 2. get the data

In [13]:
label = scio.loadmat(os.path.join(file_path, 'label.mat'))['label']
label = list(label[0])
print(label)

[1, 0, -1, -1, 0, 1, -1, 0, 1, 1, 0, -1, 0, 1, -1]


In [14]:
train_data = []
valid_data = []
test_data = []
train_label = []
valid_label = []
test_label = []

segment_length = 12 * 200 * 2
idx = np.arange(1, 401, 2)

for file in train_file:
    data = scio.loadmat(os.path.join(file_path, file))
    for key in data.keys():
        if key[:2] == '__':
            continue
        film_idx = int(key.split('_')[1][3:])
        # range from 1-15
        film_label = label[film_idx-1] + 1
        film_data = data[key]
        segment_num = film_data.shape[1] // segment_length
        filter_data = film_data[:,:segment_num*segment_length].reshape(62, segment_num, 12, 400)
        filter_data = filter_data[:,:,:,idx]
        
        filter_data = filter_data.transpose((1,0,2,3))
        filter_data = filter_data.transpose((0,2,1,3))
        filter_label = np.full((filter_data.shape[0], 1), film_label, dtype=np.float32)
        
        train_data.append(filter_data)
        train_label.append(filter_label)
    
for file in valid_file:
    data = scio.loadmat(os.path.join(file_path, file))
    for key in data.keys():
        if key[:2] == '__':
            continue
        film_idx = int(key.split('_')[1][3:])
        # range from 1-15
        film_label = label[film_idx-1] + 1
        film_data = data[key]
        segment_num = film_data.shape[1] // segment_length
        filter_data = film_data[:,:segment_num*segment_length].reshape(62, segment_num, 12, 400)
        filter_data = filter_data[:,:,:,idx]
        
        filter_data = filter_data.transpose((1,0,2,3))
        filter_data = filter_data.transpose((0,2,1,3))
        filter_label = np.full((filter_data.shape[0], 1), film_label, dtype=np.float32)
        
        valid_data.append(filter_data)
        valid_label.append(filter_label)
        

for file in test_file:
    data = scio.loadmat(os.path.join(file_path, file))
    for key in data.keys():
        if key[:2] == '__':
            continue
        film_idx = int(key.split('_')[1][3:])
        # range from 1-15
        film_label = label[film_idx-1] + 1
        film_data = data[key]
        segment_num = film_data.shape[1] // segment_length
        filter_data = film_data[:,:segment_num*segment_length].reshape(62, segment_num, 12, 400)
        filter_data = filter_data[:,:,:,idx]
        
        filter_data = filter_data.transpose((1,0,2,3))
        filter_data = filter_data.transpose((0,2,1,3))
        filter_label = np.full((filter_data.shape[0], 1), film_label, dtype=np.float32)
        
        test_data.append(filter_data)
        test_label.append(filter_label)

In [15]:
train_data = np.concatenate(train_data, axis=0)
train_label = np.concatenate(train_label, axis=0)

valid_data = np.concatenate(valid_data, axis=0)
valid_label = np.concatenate(valid_label, axis=0)

test_data = np.concatenate(test_data, axis=0)
test_label = np.concatenate(test_label, axis=0)

print(train_data.shape, train_label.shape)
print(valid_data.shape, valid_label.shape)
print(test_data.shape, test_label.shape)

(3564, 12, 62, 200) (3564, 1)
(1188, 12, 62, 200) (1188, 1)
(1188, 12, 62, 200) (1188, 1)


In [16]:
print(train_data.shape, train_label.shape)
print(valid_data.shape, valid_label.shape)
print(test_data.shape, test_label.shape)

(3564, 12, 62, 200) (3564, 1)
(1188, 12, 62, 200) (1188, 1)
(1188, 12, 62, 200) (1188, 1)


## SSL:
sample 3500 segments from train_data, 80% for training and 20% for validation
1. 2800 * 12 * 62 * 200
2. 700 * 12 * 62 * 200

## Downstream:
sample another 2000 segments from train_data

sample 500 segments from valid_data

use whole test_data for test set
1. 2000 * 12 * 62 * 200
2. 500 * 12 * 62 * 200
3. 1188 * 12 * 62 * 200

In [17]:
ssl_idx = np.random.choice(train_data.shape[0], 3500, replace=False)
ssl_train_idx = ssl_idx[:2800]
ssl_val_idx = ssl_idx[-700:]

train_idx = np.random.choice(train_data.shape[0], 2000, replace=False)
val_idx = np.random.choice(valid_data.shape[0], 500, replace=False)
test_idx = np.random.choice(test_data.shape[0], test_data.shape[0], replace=False)
# shuffle test_idx

In [18]:
ssl_train_data = train_data[ssl_train_idx]
ssl_val_data = train_data[ssl_val_idx]

train_data = train_data[train_idx]
train_label = train_label[train_idx]

val_data = valid_data[val_idx]
val_label = valid_label[val_idx]

test_data = test_data[test_idx]
test_label = test_label[test_idx]

In [20]:
save_dir = '/data/EEG_Emotion_Recognition/'

np.savez_compressed(os.path.join(save_dir, 'ssl_train_data.npz'), ssl_train_data)
np.savez_compressed(os.path.join(save_dir, 'ssl_val_data.npz'), ssl_val_data)

np.savez_compressed(os.path.join(save_dir, 'train_data.npz'), train_data)
np.savez_compressed(os.path.join(save_dir, 'train_label.npz'), train_label)

np.savez_compressed(os.path.join(save_dir, 'val_data.npz'), val_data)
np.savez_compressed(os.path.join(save_dir, 'val_label.npz'), val_label)

np.savez_compressed(os.path.join(save_dir, 'test_data.npz'), test_data)
np.savez_compressed(os.path.join(save_dir, 'test_label.npz'), test_label)

In [21]:
save_dir_2 = '/data/EEG_Emotion_Recognition/index/'

np.savez_compressed(os.path.join(save_dir_2, 'ssl_train_idx.npz'), ssl_train_idx)
np.savez_compressed(os.path.join(save_dir_2, 'ssl_val_idx.npz'), ssl_val_idx)
np.savez_compressed(os.path.join(save_dir_2, 'train_idx.npz'), train_idx)
np.savez_compressed(os.path.join(save_dir_2, 'val_idx.npz'), val_idx)
np.savez_compressed(os.path.join(save_dir_2, 'test_idx.npz'), test_idx)